In [1]:
import pandas as pd
import modules as mod
import importlib

flights = mod.inputs.flights
tail_numbers = mod.inputs.tail_numbers

c:\Users\u222152\OneDrive - British Airways Plc\Visa and Personal\Personal Projects\flight-delays\modules\inputs.py:5: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv('Inputs/flights.csv')


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
flights_with_models = flights.merge(tail_numbers, left_on='TAIL_NUMBER', right_on = 'Tail Number', how='left')

In [4]:
flights_with_models = flights_with_models[flights_with_models['Aircraft Model'].notna()]

In [5]:
flights_with_models = flights_with_models.drop(columns=['Tail Number'])
flights_with_models = flights_with_models.rename(columns={'Aircraft Model': 'AIRCRAFT_MODEL'})

In [6]:
sampled_flights = flights_with_models.groupby('AIRLINE', group_keys=False).apply(lambda x: x.sample(frac=0.05, random_state=42))

In [ ]:
"""""

def run_delay_prediction_analysis(flight_data):

    predictor = mod.models.FlightDelayPredictor(flight_data)

    processed_data = predictor.preprocess_data()

    print("\n" + "="*60)
    print("TRAINING MODELS FOR DEPARTURE DELAY PREDICTION")
    print("="*60)
    results = predictor.train_models(target='ARRIVAL_DELAYED')

    best_model_name, best_model = predictor.evaluate_best_model()
    
    print(f"\n\nPerforming hyperparameter tuning for {best_model_name}...")
    tuned_model = predictor.hyperparameter_tuning(best_model_name)
    
    return predictor, results, tuned_model

predictor, results, tuned_model = run_delay_prediction_analysis(sampled_flights)

"""

In [ ]:
importlib.reload(mod)

In [7]:
def preprocess_data(flight_data):
    predictor = mod.models.FlightDelayPredictor(flight_data)
    processed_data = predictor.preprocess_data()
    return processed_data, predictor

def run_models(predictor, target):

    print("\n" + "="*60)
    print(f"TRAINING MODELS FOR {target} PREDICTION")
    print("="*60)
    results = predictor.train_models(target)

    best_model_name, best_model = predictor.evaluate_best_model()
    return results, best_model_name, best_model

def run_hyperparameter_tuning(predictor, best_model_name):
    print(f"\n\nPerforming hyperparameter tuning for {best_model_name}...")
    tuned_model = predictor.hyperparameter_tuning(best_model_name)
    return tuned_model

In [8]:
preprocessed_data, predictor = preprocess_data(sampled_flights)
results, best_model_name, best_model = run_models(predictor, target='DEPARTURE_DELAYED')
#tuned_model = run_hyperparameter_tuning(predictor, best_model_name)

Starting data preprocessing...
Data shape after preprocessing: (205878, 71)

TRAINING MODELS FOR DEPARTURE_DELAYED PREDICTION
Using 17 features for prediction
Final feature matrix shape: (205878, 139)
Target distribution: {0: 169482, 1: 36396}

Training Logistic Regression...
AUC Score: 0.6784
CV Score: 0.6758 (+/- 0.0044)
sum of importance: 7.366432620117856

Top 20 Most Important Features:
                         feature  importance
6                 SCHEDULED_TIME    0.389914
4       SCHEDULED_DEPARTURE_HOUR    0.382780
7                       DISTANCE    0.361813
125  AIRCRAFT_MODEL_top_B737-700    0.181796
122  AIRCRAFT_MODEL_top_B737-300    0.172039
28                AIRLINE_top_AS    0.150461
5         SCHEDULED_ARRIVAL_HOUR    0.144282
131   AIRCRAFT_MODEL_top_CRJ-200    0.128389
52        ORIGIN_AIRPORT_top_ORD    0.126537
72   DESTINATION_AIRPORT_top_ORD    0.119940
15                       MONTH_9    0.111464
30                AIRLINE_top_DL    0.103618
77   DESTINATION_AIR

KeyboardInterrupt: 